In [ ]:
import numpy as np
from scipy.linalg import eigh, inv, pinvh
from nupack4 import *

from IPython import display

def dynamic(value):
    display.clear_output(wait=True)
    display.display(value)
    
import pandas as pd

def table(**kwargs):
    for k, v in kwargs.items():
        x = round(np.log10(np.max(np.abs(v)))/6)*6
        print('\t'+k, '\t1e%+d' % x)
        print(pd.DataFrame(v * 10**-x))
        print()
        
# Define the model used for all the states (temperature and gt-pairs, dangles always have to be none)
model = Model(dangles='none', gt=False)
model.unimolecular_scaling /= 1e0
model.bimolecular_scaling /= 1e0
model.molarity = 1e-7
engine = Engine(model=model)
count = Engine(model=model, count=True)
state = lambda s, dp=None: State(s, dp, kind='slow', model=model)

join_scale = model.bimolecular_scaling * model.molarity / model.unimolecular_scaling / 10

In [ ]:
# System defined
#pf, commute, sequences, dps = 0, 0, 'GCGUCGCGUCGCUAUGC', ['.....((((....))))', '((((.((...)).))))', '(((......))).....']
#sequences, dps = 'CAGTGCTAGCTAGCTAGCTAGCTC', []
#dps, sequences = [], 'AGTCTAGGATTCGGCGTGGGTTAACACGCCGAATCCTAGACTACTTTG+AGTCTAGGATTCGGCGTGGGTTAA+TTAACCCACGCCGAATCCTAGACTCAAAGTAGTCTAGGATTCGGCGTG'
#dps, sequences = ['.....((((((........)))))).......', '................................'], 'CTAGTCGATCGACTGATCACGATCGTAGCTTA'
#pf, commute, dps, sequences = None, None, ['.....((((........))))......(((...)))', '(((((......)))))..(((......)))......'], 'GUUGGCCAGAACCAACGCUGGGUUGCACCAAUGUGG'
#pf, commute, dps, sequences = 5.020713978e+10, 0.04825860176, ['((((((((...........))))))))', '........((((((((...))))))))'], 'CGGCGGCGCGGCGGCGAAACGCCGCCG' 
pf, commute, dps, sequences = 91304071.803, 0.04825860176, ['((((((.........))))))', '......((((((...))))))'], 'GGCCGGGGCCGGAAACCGGCC' 

#pf, commute, dps, sequences = 91304071.803, 0.04825860176, ['(((((........)))))', '.....(((((...)))))'], 'GCCGGGCCGGAAACCGGC' 

#pf, commute, dps, sequences = 0, 0, ['((((....))))', '(((......)))', '............'], 'AAAAATTTTTTT'
#sequences, dps = 'GUCGCGUCGCGUCGCUAUGCGAC', ['..(((...)))((((...)))).', '...((...))(((((...)))))', '..........(((((...)))))', '(((((((.((...)).)))))))','.....((((((......))))))']

#dps = ['((...))..........','(((......))).....','((.(.((...)).).))']

unpaired = state(sequences)
state_with = lambda dp: unpaired.with_structure(dp)

#w = State('AUCUGGUCCCAAGUUUCCCCGAUCGUAGGAUCGGCCCAAAUUAACAGGGUCACUCAUCGGAGUGAGGAAACAUCCAGCCGGAGGUCCGGCUGGGGGACCAGU',kind='slow',model=mod)

pfs = engine.partitions(sequences, ordered=True)
sampler = complex_sampler(sequences, pfs, join_scale)
    
total_pf = sum(i[1] for i in sampler.weights)
print('Total PF: %.4e should be %.4e' % (total_pf, pf))

bstates = list(map(state_with, dps))

while len(bstates) < 16:
    w = state_with(sampler(engine, 1)[0])
    if not bstates or min(w ^ b for b in bstates) > 0*len(sequences)/5 :
        print(len(bstates), len(w.complexes.complex_indices), '%.2f' % w.energy, w.dp, sep='\t')
        bstates.append(w)
        
bstates = sorted(bstates)

basis = [y.pairs for y in bstates]

find_dp = lambda x: next(i for i, p in enumerate(basis) if p.dp([len(p)])[1:-1] == x)
bcs = tuple(map(find_dp, dps))
print('BCs:', bcs)

launches = sampler(engine, 800)

decay = 1 * 1e-6
t =    50 * 1e-6
print('\nSimulation timescale: \t%e\nDecay timescale: \t%e' % (t, decay))

obs = hamming_observer(basis, bcs, 2, 0)
hamming = lambda: hamming_integrator(obs, 1/decay, 0e-5, decay*10)

local = hamming_integrator()
def add_one(i, launch, n=len(launches) // 100):
    if (i+1) % n == 0: dynamic(i+1)
    ham = hamming()
    stuff = state_with(launch).run(time=t, observers=[ham])
    local.add(ham)

In [ ]:
def speed(state, *obs):
    return state.run(walltime=0.2, observers=[*obs, timer()])[-1].time / 0.2

print('speed', speed(state_with(launches[0])))
sp = speed(state_with(launches[0]), hamming())
print('speed', sp)
eta = len(launches) * t / 8 / sp
print('ETA: %s after %.2f minutes' % (time_string(delta=eta), eta / 60))

In [ ]:
thread_map(launches, add_one, index=True);

In [ ]:
print('Total simulation time: %e' % local.time)
A = slice(-len(bcs), None)

bc_part = np.arange(len(local.times)) >= len(local.times) - len(bcs)
big_enough = np.abs(local.times / local.time) > 0e-1
O = np.where(big_enough | bc_part)[0]

mu = local.average_occupation()[O]
Overlap = local.average_overlap()[:, O][O]
Gamma = local.average_gamma()[:, O][O]
Grad = local.average_gradient()[:, O][O]

p0, G0 = projected_G(Gamma[:, A][A], mu[A])
S0 = S_from_G(G0, mu[A])
H0 = H_from_S(S0)

scale = abs(min(0., eigh(np.diag(1/mu) @ (Gamma - Grad / decay) @ np.diag(1/mu))[0].min()))
print('Diagonal noise:', scale)
noise = scale * np.diag(mu*mu)
noise = 75.0 * decay * decay * Overlap / local.time
#noise = Grad / decay * 6e-7
Gamma1 = (Gamma @ pinvh(Gamma - Grad / decay + noise) @ Gamma)[:, A][A]

p1, G1 = projected_G(Gamma1, mu[A])
S1 = S_from_G(G1, mu[A])
H1 = H_from_S(S1)

table(H0=H0, p0=p0)
table(H1=H1, p1=p1)

In [ ]:
if True:
    all_states, pi, R = unpaired.rate_matrix(sparse=False)
    S = S_from_R(R, pi, 0)
    indices = [all_states.index(bstates[b]) for b in bcs]
    Se = S[:, indices][indices]
    He = H_from_S(Se)
    table(He=He)
    
    B = np.asarray([obs(w.pairs) for w in all_states])
    PB = np.diag(pi) @ B
    mud = B.T @ pi
    Sd = S_from_R(R, pi, 1/decay)
    Gd = PB.T @ Sd @ PB
        
    Sd0 = S_from_G(Gd, mud)[:, A][A]
    Hd0 = H_from_S(Sd0)
    table(Hd0=Hd0)
    
    Grad_E = PB.T @ Sd @ np.diag(pi) @ Sd @ PB
    Gd1 = Gd @ pinv(Gd - Grad_E / decay) @ Gd

    Sd1 = S_from_G(Gd1[:, A][A], mud[A])
    Hd1 = H_from_S(Sd1)
    table(Hd1=Hd1)

In [ ]:
if False:
    t2 = 2.
    def hits(w, pairs):
        obs = hitting_time_observer(pairs)
        w.run(time=t2, observers=[obs])
        return obs.samples, obs.numbers, obs

    starts = bstates
    
    data = thread_map(starts, lambda w: hits(w, basis))
    samples = sum(d[0] for d in data)
    numbers = sum(d[1] for d in data)
    samples = samples / np.maximum(1, numbers)
    Hs = samples - np.diag(np.diagonal(samples))

In [ ]:
def hit(x, y):
    return x.run(first_collision(y.pairs), observers=[timer()])[-1].time

Hs = np.zeros([len(bcs), len(bcs)])
Hp = np.zeros([len(bcs), len(bcs)])

if False:
    for i in range(len(bcs)):
        for j in range(len(bcs)):
            if i != j:
                data = np.asarray(thread_map(48, lambda _: hit(bstates[bcs[i]], bstates[bcs[j]])))
                Hs[i, j] = data.mean()
                Hp[i, j] = data.std() * len(data)**-0.5

In [ ]:
pfs = np.asarray([bstates[b].boltz for b in bcs])
print('Probabilities')
print(p0)
print(p1)
print(pfs / pfs.sum())

In [ ]:
n = 10
table(G=Gamma[n:n+4,n:n+4], G2=Gd[n:n+4, n:n+4])

In [ ]:
if False:
    pa = 0 * pi
    for l in sampler(engine, 8000):
        pa[all_states.index(state_with(l))] += 1
    pa /= pa.sum()
    print(pa)

In [ ]:
eigh(np.diag(np.diagonal(Overlap)**-0.5) @ Overlap @ np.diag(np.diagonal(Overlap)**-0.5))[0]